In [10]:
from tqdm import tqdm
from requests import request
import pandas as pd

In [11]:
energy_consumption_dataset = pd.DataFrame()

In [12]:
res = request("GET", 'https://www.smard.de/app/chart_data/410/DE/index_hour.json')
timestamps = res.json()["timestamps"]

In [13]:
time_series = []
for timestamp in tqdm(timestamps[:20]):
    res = request("GET", "https://www.smard.de/app/chart_data/410/DE/410_DE_hour_" +str(timestamp)+".json")
    body = res.json()["series"]
    time_series += body

100%|██████████| 20/20 [00:02<00:00,  8.68it/s]


In [32]:
energy_consumption = pd.DataFrame(time_series)
energy_consumption.index = [pd.Timestamp(x, unit='ms') for x in energy_consumption.iloc[:, 0]]
energy_consumption = energy_consumption.drop(columns=0)
energy_consumption = energy_consumption.dropna()
energy_consumption.columns = ["Energy Consumption"]
energy_consumption.head()

,Energy Consumption
2014-12-31 23:00:00,44600.25
2015-01-01 00:00:00,43454.75
2015-01-01 01:00:00,41963.25
2015-01-01 02:00:00,40617.75
2015-01-01 03:00:00,39936.75


In [14]:
wind_data = pd.read_csv("produkt_f_stunde_19750701_20221231_03987.txt", sep=";")
solar_data = pd.read_csv("produkt_st_stunde_19451231_20230930_03987.txt", sep=";")
temperature_data = pd.read_csv("produkt_tu_stunde_18930101_20221231_03987.txt", sep=";")
precipitation_data = pd.read_csv("produkt_rr_stunde_19950901_20221231_03987.txt", sep=";")

In [15]:
wind_data =
wind_data.head()

,STATIONS_ID,MESS_DATUM,QN_8,FF,DD,eor
0,3987,1975070100,1,5.0,330,eor
1,3987,1975070106,1,3.0,340,eor
2,3987,1975070112,1,6.0,330,eor
3,3987,1975070118,1,7.0,360,eor
4,3987,1975070200,1,4.0,340,eor


In [16]:
solar_data.head()

,STATIONS_ID,MESS_DATUM,QN_592,ATMO_LBERG,FD_LBERG,FG_LBERG,SD_LBERG,ZENIT,MESS_DATUM_WOZ,eor
0,3987,1945123101:10,1,-999.0,0.0,0.0,0,146.08,1945123102:00,eor
1,3987,1945123102:10,1,-999.0,0.0,0.0,0,139.16,1945123103:00,eor
2,3987,1945123103:10,1,-999.0,0.0,0.0,0,130.76,1945123104:00,eor
3,3987,1945123104:10,1,-999.0,0.0,0.0,0,121.73,1945123105:00,eor
4,3987,1945123105:10,1,-999.0,0.0,0.0,0,112.61,1945123106:00,eor


In [17]:
temperature_data.head()

,STATIONS_ID,MESS_DATUM,QN_9,TT_TU,RF_TU,eor
0,3987,1893010101,5,-12.3,84.0,eor
1,3987,1893010102,5,-10.9,84.0,eor
2,3987,1893010103,5,-9.9,84.0,eor
3,3987,1893010104,5,-9.4,84.0,eor
4,3987,1893010105,5,-8.9,84.0,eor


In [18]:
precipitation_data.head()

,STATIONS_ID,MESS_DATUM,QN_8,R1,RS_IND,WRTR,eor
0,3987,1995090100,1,1.3,1,-999,eor
1,3987,1995090101,1,1.7,1,-999,eor
2,3987,1995090102,1,0.5,1,-999,eor
3,3987,1995090103,1,0.5,1,-999,eor
4,3987,1995090104,1,0.9,1,-999,eor
